In [209]:
import pandas as pd

tabula = pd.read_csv('./arabic_media_dictionary.csv')
tabula.columns = ['context', 'translation', 'arabic', 'a']
tabula['page'] = (tabula.context == 'Contextualisation of Arabic entries').cumsum()
tabula = tabula[tabula.context != 'Contextualisation of Arabic entries']

displaced_pages = tabula.groupby('page')['a'].count()[lambda x: x>0].index.values

tabula.translation = tabula.translation.mask(tabula.page.isin(displaced_pages), tabula.arabic)
tabula.arabic = tabula.arabic.mask(tabula.page.isin(displaced_pages), tabula.a)

tabula = tabula[['context', 'translation', 'arabic', 'page']]

tabula['letter'] = tabula.context.where(tabula.context.str.len() == 1).fillna(method='ffill')

tabula = tabula[tabula.context != tabula.letter].reset_index(drop=True)

In [ ]:
def get_first_arabic_letter(s):
    if s=='':
        return None
    if len(s) > 2:
        if s[:2] == 'ال':
            return s[2]
    return s[0]

tabula['first_arabic_letter'] = tabula.arabic.fillna('').apply(get_first_arabic_letter)
tabula['starts_with_al'] = tabula.arabic.fillna('').apply(lambda x: x[:2] == 'ال' if len(x)>1 else None)

In [210]:
translation_lower_case = ((tabula.translation.notna())
                         &(tabula.translation.fillna('').apply(lambda s: s[0].islower() if s != '' else False)))
no_blanks_in_row = tabula[['context','translation','arabic']].isna().sum(axis=1) == 0
blanks_in_row = ~no_blanks_in_row

is_continuation = translation_lower_case | blanks_in_row
is_new_entry = ~is_continuation

tabula['entry_number'] = is_new_entry.cumsum()

tabula

,context,translation,arabic,page,letter,entry_number
0,بعد بحث طويل دام سنوات استطاع خالد العثور على ...,Father,أبٌَ (ج( آباءٌ,0,ا,1
1,الشرعي والذي تركه وأمه وهو صغير السن لتقوم بتر...,NaN,NaN,0,ا,1
2,أسرة أخرى.,NaN,NaN,0,ا,1
3,يحرص أهل جبال الأطلس المغربيةّ على عادات وتقاليد,Handed down from,أبا ًعن جد,0,ا,2
4,توارثوها أبا ًعن جدّ,father to son,NaN,0,ا,2
...,...,...,...,...,...,...
14193,الأجنبي,NaN,NaN,360,ي,6647
14194,وكان السادس من تشرين الأوّل يوم الهجوم من قبِل...,D-day,يوَْمُ الهجُُومِ,360,ي,6648
14195,السوريّ والمصريّ على إسرائيل في حرب تشرين,NaN,NaN,360,ي,6648
14196,لم يفلح الصحفي في الحصول على موعد للقاء الرئيس...,Diary; calendar,يوَْمِية,360,ي,6649


In [243]:
def compile_entry(df):
    return pd.Series({c: ' '.join(df[df[c].notna()][c].values) for c in df.columns})

entries = tabula.groupby('entry_number')[['context','translation','arabic']].apply(compile_entry)
entries.sample(20)

,context,translation,arabic
entry_number,,,
3596,ويبقى ملف الحرب على العراق شديد الحساسية في بر...,Very sensitive,شَدِيدُ الحَسَاسِيةَِ
2707,غادر الحيّ الذي نشأ فيه إلى الخارج لإتمام دراسته,Neighbourhood,حَي (ج( أحَْياءٌ
3082,بعد طفرة النفط تراكم لدى بعض البلدان العربيةّ ...,Liquid capital,رَأْسُمَالٍ سَائلِ
5178,الأوروبيةّ,Council,الخَليِجِي
4011,أشار وزير الصحة البريطاني إلى ضرورة اتخاذ خطوا...,Urgent; pressing,عَاجِلٌ
964,قال المتحدث الرسمي إنّ تحليق طيران العدو في أج...,Violation of sovereignty,انِْتهِاَكُ السِياَدَةِ
4970,استمرار مساهمة الصندوق العربي في دعم برنامج ال...,Digital language,لغَُة ٌرَقمَِية
4298,العديد من الدول لها عيدٌ وطني تحتفل به سنويا و...,National day,عِيدٌ وَطنيِ
4380,وافقت المعارضة ُعلى عقد لقاء غير مشروط مع الحك...,Unconditional,غَيْرُ مَشْرُوطٍ


In [246]:
tabula[11100:11106]

,context,translation,arabic,page,letter,entry_number
11100,الجديدة للمؤسسة,NaN,NaN,282,م,5175
11101,ولا بد لأي قرارٍ جديد في سياسة تحريرِ المجلةّ ...,The editorial board,مَجْلسُِ التحّْرِيرِ,282,م,5176
11102,على توصية مجلس التحرير,NaN,NaN,282,م,5176
11103,عقد مجلس التعاون الخليجي اتفاقية شراكة مع بعض ...,Gulf Cooperation,مَجْلسُِ التعَاوُنِ,282,م,5177
11104,الأوروبيةّ,Council,الخَليِجِي,282,م,5178
11105,وشكل عام 1649 منعطفا في تاريخ بريطانيا الحديث ...,The House of Commons,مَجْلسُِ العُمُومِ,282,م,5179


In [244]:
entries.sample(20)

,context,translation,arabic
entry_number,,,
2522,صرّح رئيس الشركة أنّ الحساب مفتوح لتطوير الشرك...,Open account,حِسَابٌ مَفْتوحٌ
4664,ويمكن مراجعة قسم القبول في الجامعة لمعرفة الأو...,Admission,قبَوُلٌ
3562,بدا الرئيس شارد البال في المقابلة الصحفيةّ الت...,Confused; disconcerted,شَارِدُ الباَلِ
4320,وسيتَعََرض المخالفون إلى غرامة فاحشة فيما لو خ...,Excessive fine,غَرَامَة ٌفاَحِشَة
6413,كان هدفه المنشود هو الحصول على درجة الدكتوراه,The intended goal,الهدََفُ المَنْشودُ
5483,تحاول بعض جمعيات حقوق الإنسان الدفاع عن المستض...,Oppressed,مُسْتضَْعَفٌ
2838,و يخوّل القرار لمجلس الأمن استعمال القوّة عندم...,To authorise; to empower; to grant; to entitle,خَولَ – تخَْوِيلٌ
66,يتقن هذا الطالب أربع لغات إلى جانب اللغة الأم,To master; to command,أتَْقنََ
6514,أصدرت الحكومة أوراقا نقديةّ جديدة وفيها صورة ا...,Bank note,وَرَقةَ ٌنقَْدِية


In [191]:
(tabula[(tabula.first_arabic_letter.notna())
       &(tabula.first_arabic_letter != tabula.letter)]
       .groupby(['letter', 'first_arabic_letter'])
       [['letter']].count()
       .rename(columns={'letter': 'count'})
       .sort_values('count', ascending=False))[:20]

count
letter first_arabic_letter       
ا      أ                      433
       إ                      191
م      ا                       66
ت      ا                       41
ح      ا                       28
ب      ا                       22
ع      ا                       22
س      ا                       21
ق      ا                       17
ا      آ                       16
ن      ا                       16
و      ا                       14
ج      ا                       14
ض      ا                        9
د      ا                        9
ف      ا                        9
ش      ا                        8
ا      و                        7
ص      ا                        7
ا      م                        6

In [193]:
tabula[(tabula.letter=='ا')&(tabula.first_arabic_letter=='آ')]

,context,translation,arabic,page,letter,first_arabic_letter,starts_with_al
10,...(( وتشير الدراسة إلى تخفيف ومنع الضّرر البي...,Oil wells,آبارُ النفْطِ,0,ا,آ,False
166,بعد فحص دقيق خلص الأطباء إلى نتيجة مفادها أن هذا,Side effect,آثارٌ جانبِيِة,4,ا,آ,False
492,تشير آخر الأخبار إلى تفوق حزب المحافظين البريط...,Latest news,آخِرُ الَأخْباَرِ,12,ا,آ,False
494,ومعنا مراسلنا من القاهرة ليوافينا بآخر التطورا...,Latest development,آخِرُ التطوراتِ,12,ا,آ,False
604,آذنت الأزمة المالية في الفيفا بسقوط رئيسها,To show signs of,آذِنَ باِلسُقوطِ,15,ا,آ,False
606,آذى قرار الحرب على العراق عام 2003 وحدة البلاد,To harm; to hurt,آذَى,15,ا,آ,False
608,آذى كلامه النابي مشاعر الحاضرين، ما جعل البعض ...,To hurt somebody’s,آذَى مَشاعِرَ شَخْصٍ,15,ا,آ,False
684,آزرت منظمات إنسانية النازحين من مناطق النزاع وذلك,To help; to support,آزَرَ,17,ا,آ,False
1104,تربطهم آصرة القرابة ولهذا ظلوا في صلة ببعضهم ا...,Bond; tie,آصِرَة,27,ا,آ,False
1771,آل به المطاف إلى الشارع لأنه بذّر كل ثروته في ...,He eventually got to the,آلَ بهِ المَطافُ إلِىَ,44,ا,آ,False


In [195]:
tabula.groupby('letter').starts_with_al.value_counts().unstack()

starts_with_al,False,True
letter,,
ا,1143.0,78.0
ب,336.0,22.0
ت,653.0,41.0
ث,31.0,1.0
ج,155.0,14.0
ح,360.0,28.0
خ,140.0,4.0
د,186.0,9.0
ذ,25.0,3.0


In [175]:
tabula[(tabula.first_arabic_letter=='ن')]

,context,translation,arabic,page,letter,first_arabic_letter
906,استقرت الأمور في نصابها اليوم وعاد الجميع إلى ...,NaN,نصِابهِا,22,ا,ن
2955,استكشافيةّ جنوبي البصرة بعمق يبلغ حوالي 6 آلاف...,NaN,نفَْطِية,74,ب,ن
4235,القطاع الخاص والعام، داعية إلى أخذ خطوات ملموس...,discrimination,نوَْعِ الجِنْسِ,106,ت,ن
4758,نوع الجنس هو مجموع الصفات المتعلقة والمميزة ما...,Gender,نوَْعُ الجِنْسِ,119,ج,ن
8206,بعض الأمور إلى نصابها وتمّ رفع الحظر الاقتصادي عن,normal,نصَِابهِاَ,205,ع,ن
11509,قال مسؤولون محليوّن إنّ مسلحين قتلوا خمسة جنود في,Checkpoint,نقُْطة ُتفَْتيِشٍ,288,م,ن
11511,يذكر أنّ البنك المركزي الأوروبي كان قد رفع خلال,Percentage point,نقُْطة ٌمِئوَِية,288,م,ن
11515,نقل السّجين من السّجن المحلي إلى السّجن المركز...,Transfer,نقَْلٌ,288,م,ن
11517,تم تنظيم وقفة احتجاجية أمام مقرّ هيئة الإذاعة ...,Transfer of news,نقَْلُ الخَبرَِ,288,م,ن
11520,واحتاج المريض إلى نقل دم بصورة عاجلة نظرا لفقد...,Blood transfusion,نقَْلُ الدمِ,288,م,ن


In [176]:
tabula[(tabula.letter=='ا')&(tabula.arabic.notna())].arabic.fillna('').apply(lambda x: x[0]).value_counts()

#alephs = ['ا', 'أ', 'إ', 'آ']

ا    484
أ    402
إ    179
آ     14
و      7
م      5
س      2
ع      2
ب      1
ص      1
ف      1
(      1
ل      1
ط      1
ن      1
ج      1
خ      1
Name: arabic, dtype: int64

In [177]:
tabula[tabula.translation.str.strip()=='Flexible budget']

,context,translation,arabic,page,letter,first_arabic_letter
